In [1]:
import pandas as pd
import pickle

path = 'data/gamecodes.data'
with open(path, 'rb') as f:
    gcodes = pickle.load(f)

In [2]:
teams = {'Atlanta Falcons':'atl','Buffalo Bills':'buf','Carolina Panthers':'car','Chicago Bears':'chi',
         'Cincinnati Bengals':'cin','Cleveland Browns':'cle','Indianapolis Colts':'clt',
         'Arizona Cardinals':'crd','Dallas Cowboys':'dal','Denver Broncos':'den','Detroit Lions':'det',
         'Green Bay Packers':'gnb','Houston Texans':'htx','Jacksonville Jaguars':'jax',
         'Kansas City Chiefs':'kan','Miami Dolphins':'mia','Minnesota Vikings':'min','New Orleans Saints':'nor',
         'New England Patriots':'nwe','New York Giants':'nyg','New York Jets':'nyj','Tennessee Titans':'oti',
         'Philadelphia Eagles':'phi','Pittsburgh Steelers':'pit','Oakland Raiders':'rai',
         'Las Vegas Raiders':'rai','St. Louis Rams':'ram','Los Angeles Rams':'ram','Baltimore Ravens':'rav',
         'San Diego Chargers':'sdg','Los Angeles Chargers':'sdg','Seattle Seahawks':'sea',
         'San Francisco 49ers':'sfo','Tampa Bay Buccaneers':'tam','Washington Redskins':'was'}
cols = ['H_Pts','V_Pts','H_Pts_Opp','V_Pts_Opp','H_Off_Pass','V_Off_Pass','H_Off_Rush','V_Off_Rush',
        'H_Def_Pass','V_Def_Pass','H_Def_Rush','V_Def_Rush','H_TD','V_TD','H_TD_on_Def','V_TD_on_Def','H_FG_Made',
        'V_FG_Made','H_FG_Att','V_FG_Att','H_RZ_Conv','V_RZ_Conv','H_RZ_Att','V_RZ_Att','H_RZ_Def_Conv',
        'V_RZ_Def_Conv','H_RZ_Def_Att','V_RZ_Def_Att','H_Poss','V_Poss','H_Plays','V_Plays','H_TO_Gain','V_TO_Gain',
        'H_TO_Lost','V_TO_Lost','H_Yds_Pen','V_Yds_Pen','H_Sacks_Def','V_Sacks_Def','H_Tackles_Loss','V_Tackles_Loss',
        'H_Kickret','V_Kickret','H_Puntret','V_Puntret']
hdictmap = {'Points':'H_Pts','Points_Opp':'H_Pts_Opp','Yds_Off_Pass':'H_Off_Pass','Yds_Off_Rush':'H_Off_Rush',
           'Yds_Def_Pass':'H_Def_Pass','Yds_Def_Rush':'H_Def_Rush','TD':'H_TD','TD_on_Def':'H_TD_on_Def',
            'FG_Made':'H_FG_Made','FG_Att':'H_FG_Att','RZ_Conv':'H_RZ_Conv','RZ_Att':'H_RZ_Att',
            'RZ_Def_Conv':'H_RZ_Def_Conv','RZ_Def_Att':'H_RZ_Def_Att','Possession':'H_Poss','Plays':'H_Plays',
           'TO_Gained':'H_TO_Gain','TO_Lost':'H_TO_Lost','Yds_Pen':'H_Yds_Pen','Sacks_Def':'H_Sacks_Def',
           'Tackles_Loss':'H_Tackles_Loss','Yds_per_Kickret':'H_Kickret','Yds_per_Puntret':'H_Puntret'}
vdictmap = {'Points':'V_Pts','Points_Opp':'V_Pts_Opp','Yds_Off_Pass':'V_Off_Pass','Yds_Off_Rush':'V_Off_Rush',
           'Yds_Def_Pass':'V_Def_Pass','Yds_Def_Rush':'V_Def_Rush','TD':'V_TD','TD_on_Def':'V_TD_on_Def',
            'FG_Made':'V_FG_Made','FG_Att':'V_FG_Att','RZ_Conv':'V_RZ_Conv','RZ_Att':'V_RZ_Att',
            'RZ_Def_Conv':'V_RZ_Def_Conv','RZ_Def_Att':'V_RZ_Def_Att','Possession':'V_Poss','Plays':'V_Plays',
           'TO_Gained':'V_TO_Gain','TO_Lost':'V_TO_Lost','Yds_Pen':'V_Yds_Pen','Sacks_Def':'V_Sacks_Def',
           'Tackles_Loss':'V_Tackles_Loss','Yds_per_Kickret':'V_Kickret','Yds_per_Puntret':'V_Puntret'}

In [35]:
years = [str(year) for year in range(2010,2020)]
years = [years[1]]
#codes = [gcodes[i] for i in range(len(gcodes)) 
#      if (((gcodes[i][0:4] == year) & (gcodes[i][4:6] != '01')) | (gcodes[i][0:6] == (str(int(year)+1)+'01')))]

In [36]:
dfgame=pd.DataFrame(index=gcodes,columns=cols)
dfgame.index.set_names('Code',inplace=True)

In [37]:
for year in years:
    path = 'data/df_step2_'+year+'.data'
    with open(path, 'rb') as f:
        dfyear = pickle.load(f)
    codes = set([ind[1] for ind in dfyear.index])
    for code in codes:
        home = code[-3:]
        away = teams[dfyear.loc[(home,code),'Opponent']]
        for key in hdictmap:
            dfgame.loc[code,hdictmap[key]] = dfyear.loc[(home,code),key]
        for key in vdictmap:
            dfgame.loc[code,vdictmap[key]] = dfyear.loc[(away,code),key]

In [46]:
dfyear.groupby(level=0).sortlevel()

AttributeError: 'DataFrameGroupBy' object has no attribute 'sortlevel'

In [44]:
pd.MultiIndex.from_frame(dfyear).sortlevel()

(MultiIndex([('A',   'Arizona Cardinals',  0,  3, 222, 178, 271, 131, 2, ...),
             ('A',   'Arizona Cardinals',  0,  3, 226, 120, 313,  74, 2, ...),
             ('A',   'Arizona Cardinals',  0,  6, 255, 150, 222,  70, 1, ...),
             ('A',   'Arizona Cardinals',  0,  6, 299,  75, 247, 103, 1, ...),
             ('A',   'Arizona Cardinals', 19, 21, 175,  90, 284,  55, 1, ...),
             ('A',   'Arizona Cardinals', 21, 28, 422,  74, 309,  99, 4, ...),
             ('A',   'Arizona Cardinals', 31, 27, 321,  54, 237, 156, 4, ...),
             ('A',   'Arizona Cardinals', 32, 20, 361,  91, 272,  73, 3, ...),
             ('A',     'Atlanta Falcons',  3,  0, 322,  41, 351, 138, 2, ...),
             ('A',     'Atlanta Falcons', 14, 24, 186,  64, 262,  89, 2, ...),
             ...
             ('H',    'Tennessee Titans', 27, 13,  82, 205, 331,  66, 2, ...),
             ('H',    'Tennessee Titans', 38, 17, 261, 174, 262,  66, 5, ...),
             ('H', 'Washington Reds

In [45]:
dfyear

Home/Away              Opponent  Points  Points_Opp  \
Team Code                                                               
phi  201109110ram         A        St. Louis Rams      31          13   
     201109180atl         A       Atlanta Falcons      31          35   
     201109250phi         H       New York Giants      16          29   
     201110020phi         H   San Francisco 49ers      23          24   
     201110090buf         A         Buffalo Bills      24          31   
...                     ...                   ...     ...         ...   
car  201112040tam         A  Tampa Bay Buccaneers      38          19   
     201112110car         H       Atlanta Falcons      23          31   
     201112180htx         A        Houston Texans      28          13   
     201112240car         H  Tampa Bay Buccaneers      48          16   
     201201010nor         A    New Orleans Saints      17          45   

                   Yds_Off_Pass  Yds_Off_Rush  Yds_Def_Pass  Yds_Def_Rush  TD  \
Team Code                                                                       
phi  201109110ram           187           236           209           154   4   
     201109180atl           314           133           195           138   4   
     201109250phi           211           177           254           102   1   
     201110020phi           416           108           291           164   2   
     201110090buf           315           174           193           143   4   
...                         ...           ...           ...           ...  ..   
car  201112040tam           231           163           229            78   5   
     201112110car           276           157           320            86   3   
     201112180htx           160           166           212           150   4   
     201112240car           171           270           274            59   6   
     201201010nor           158           164           409           208   2   

                   TD_on_Def  ...  RZ_Def_Att  Possession  Plays  TO_Gained  \
Team Code                     ...                                             
phi  201109110ram          1  ...           4   30.233333     68          1   
     201109180atl          0  ...           5   30.050000     69          2   
     201109250phi          0  ...           3   36.850000     75          0   
     201110020phi          0  ...           3   31.133333     73          1   
     201110090buf          1  ...           4   26.366667     61          1   
...                      ...  ...         ...         ...    ...        ...   
car  201112040tam          0  ...           6   32.683333     64          1   
     201112110car          0  ...           5   29.283333     65          0   
     201112180htx          0  ...           4   32.566667     60          3   
     201112240car          0  ...           3   27.050000     51          4   
     201201010nor          0  ...           9   25.433333     52          1   

                   TO_Lost  Yds_Pen  Sacks_Def  Tackles_Loss  Yds_per_Kickret  \
Team Code                                                                       
phi  201109110ram        1       91        5.0             7             18.0   
     201109180atl        3       20        4.0             2             26.0   
     201109250phi        3       36        3.0             4             22.5   
     201110020phi        3       55        3.0             4             22.5   
     201110090buf        5       46        1.0             3             19.0   
...                    ...      ...        ...           ...              ...   
car  201112040tam        0       20        2.0             7             17.0   
     201112110car        2       50        3.0             6             17.8   
     201112180htx        0       40        2.0             5             21.0   
     201112240car        0       77        3.0             2             20.2   
     201201010

In [38]:
for t, df in dfyear.groupby(level=0):
    print(t,df)

atl                   Home/Away              Opponent  Points  Points_Opp  \
Team Code                                                               
atl  201109110chi         A         Chicago Bears      12          30   
     201109180atl         H   Philadelphia Eagles      35          31   
     201109250tam         A  Tampa Bay Buccaneers      13          16   
     201110020sea         A      Seattle Seahawks      30          28   
     201110090atl         H     Green Bay Packers      14          25   
     201110160atl         H     Carolina Panthers      31          17   
     201110230det         A         Detroit Lions      23          16   
     201111060clt         A    Indianapolis Colts      31           7   
     201111130atl         H    New Orleans Saints       0           3   
     201111200atl         H      Tennessee Titans      23          17   
     201111270atl         H     Minnesota Vikings      24          14   
     201112040htx         A        Houston Texa

jax                   Home/Away              Opponent  Points  Points_Opp  \
Team Code                                                               
jax  201109110jax         H      Tennessee Titans      16          14   
     201109180nyj         A         New York Jets       3          32   
     201109250car         A     Carolina Panthers      10          16   
     201110020jax         H    New Orleans Saints      10          23   
     201110090jax         H    Cincinnati Bengals      20          30   
     201110160pit         A   Pittsburgh Steelers      13          17   
     201110240jax         H      Baltimore Ravens      12           7   
     201110300htx         A        Houston Texans      14          24   
     201111130clt         A    Indianapolis Colts      17           3   
     201111200cle         A      Cleveland Browns      10          14   
     201111270jax         H        Houston Texans      13          20   
     201112050jax         H    San Diego Charge

rav                   Home/Away              Opponent  Points  Points_Opp  \
Team Code                                                               
rav  201109110rav         H   Pittsburgh Steelers      35           7   
     201109180oti         A      Tennessee Titans      13          26   
     201109250ram         A        St. Louis Rams      37           7   
     201110020rav         H         New York Jets      34          17   
     201110160rav         H        Houston Texans      29          14   
     201110240jax         A  Jacksonville Jaguars       7          12   
     201110300rav         H     Arizona Cardinals      30          27   
     201111060pit         A   Pittsburgh Steelers      23          20   
     201111130sea         A      Seattle Seahawks      17          22   
     201111200rav         H    Cincinnati Bengals      31          24   
     201111240rav         H   San Francisco 49ers      16           6   
     201112040cle         A      Cleveland Brow

In [30]:
dfsorted.shape

(16, 25)

In [27]:
dfyear.shape

(16, 25)

In [98]:
dfgame.head(40)

,H_Pts,V_Pts,H_Pts_Opp,V_Pts_Opp,H_Off_Pass,V_Off_Pass,H_Off_Rush,V_Off_Rush,H_Def_Pass,V_Def_Pass,...,H_Yds_Pen,V_Yds_Pen,H_Sacks_Def,V_Sacks_Def,H_Tackles_Loss,V_Tackles_Loss,H_Kickret,V_Kickret,H_Puntret,V_Puntret
Code,,,,,,,,,,,,,,,,,,,,,
201009120phi,20,27,27,20,199,188,149,132,188,199,...,80,15,3,6,6,5,23.25,31.2,7,10
201009190det,32,35,35,32,335,284,115,162,284,335,...,51,75,5,2,7,10,16.5,21.8,5,11.3333
201009260jax,3,28,28,3,105,291,139,106,291,105,...,67,84,3,7,12,2,29.5,16.75,10.3333,12.4
201010030phi,12,17,17,12,250,125,110,169,125,250,...,80,30,1,1,2,5,12,23,25,10
201010100sfo,24,27,27,24,309,253,74,125,253,309,...,65,30,4,2,4,5,36.2,25.2,4.5,11.5
201010170phi,31,17,17,31,326,250,154,65,250,326,...,78,30,3,1,4,8,9.5,23.3333,16.6667,18
201010240oti,37,19,19,37,276,231,75,99,231,276,...,37,100,1,3,2,5,3,31.5,10.75,8
201011070phi,26,24,24,26,218,294,195,62,294,218,...,125,59,3,3,5,6,18.75,14.5,2.5,0
201011150was,28,59,59,28,295,333,105,260,333,295,...,41,35,1,2,2,5,24.1429,16.5,6,9


In [97]:
dfgame.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2560 entries, 201009120phi to 201910270ram
Data columns (total 46 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   H_Pts           256 non-null    object
 1   V_Pts           256 non-null    object
 2   H_Pts_Opp       256 non-null    object
 3   V_Pts_Opp       256 non-null    object
 4   H_Off_Pass      256 non-null    object
 5   V_Off_Pass      256 non-null    object
 6   H_Off_Rush      256 non-null    object
 7   V_Off_Rush      256 non-null    object
 8   H_Def_Pass      256 non-null    object
 9   V_Def_Pass      256 non-null    object
 10  H_Def_Rush      256 non-null    object
 11  V_Def_Rush      256 non-null    object
 12  H_TD            256 non-null    object
 13  V_TD            256 non-null    object
 14  H_TD_on_Def     256 non-null    object
 15  V_TD_on_Def     256 non-null    object
 16  H_FG_Made       256 non-null    object
 17  V_FG_Made       256 non-null    object

In [ ]:
cols = ['H_Pts','V_Pts','H_Pts_Opp','V_Pts_Opp','H_Off_Pass','V_Off_Pass','H_Off_Rush','V_Off_Rush',
        'H_Def_Pass','V_Def_Pass','H_Def_Rush','V_Def_Rush','H_TD','V_TD','H_TD_on_Def','V_TD_on_Def','H_FG_Made',
        'V_FG_Made','H_FG_Att','V_FG_Att','H_RZ_Conv','V_RZ_Conv','H_RZ_Att','V_RZ_Att','H_RZ_Def_Conv',
        'V_RZ_Def_Conv','H_RZ_Def_Att','V_RZ_Def_Att','H_Poss','V_Poss','H_Plays','V_Plays','H_TO_Gain','V_TO_Gain',
        'H_TO_Lost','V_TO_Lost','H_Yds_Pen','V_Yds_Pen','H_Sacks_Def','V_Sacks_Def','H_Tackles_Loss','V_Tackles_Loss',
        'H_Kickret','V_Kickret','H_Puntret','V_Puntret']

In [96]:
intcols=['H_Pts','V_Pts','H_Pts_Opp','V_Pts_Opp','H_Off_Pass','V_Off_Pass','H_Off_Rush','V_Off_Rush','H_Def_Pass',
        'V_Def_Pass','H_Def_Rush','V_Def_Rush','H_TD','V_TD','H_TD_on_Def','V_TD_on_Def','H_FG_Made','V_FG_Made',
        'H_FG_Att','V_FG_Att','H_RZ_Conv','V_RZ_Conv','H_RZ_Att','V_RZ_Att','H_RZ_Def_Conv','V_RZ_Def_Conv',
        'H_RZ_Def_Att','V_RZ_Def_Att','H_Plays','V_Plays','H_TO_Gain','V_TO_Gain','H_TO_Lost','V_TO_Lost',
        'H_Yds_Pen','V_Yds_Pen','H_Tackles_Loss','V_Tackles_Loss']
fltcols=['H_Poss','V_Poss','H_Sacks_Def','V_Sacks_Def','H_Kickret','V_Kickret','H_Puntret','V_Puntret']
for colu in intcols: dfgame[colu] = dfgame[colu].astype('int64')
for colu in fltcols: dfgame[colu] = dfgame[colu].astype('float64')

ValueError: cannot convert float NaN to integer

In [43]:
dfyear.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 512 entries, ('phi', '201009120phi') to ('car', '201101020atl')
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Home/Away        512 non-null    object 
 1   Opponent         512 non-null    object 
 2   Points           512 non-null    int64  
 3   Points_Opp       512 non-null    int64  
 4   Yds_Off_Pass     512 non-null    int64  
 5   Yds_Off_Rush     512 non-null    int64  
 6   Yds_Def_Pass     512 non-null    int64  
 7   Yds_Def_Rush     512 non-null    int64  
 8   TD               512 non-null    int64  
 9   TD_on_Def        512 non-null    int64  
 10  FG_Made          512 non-null    int64  
 11  FG_Att           512 non-null    int64  
 12  RZ_Conv          512 non-null    int64  
 13  RZ_Att           512 non-null    int64  
 14  RZ_Def_Conv      512 non-null    int64  
 15  RZ_Def_Att       512 non-null    int64  
 16  Possession       512